In [1]:
from ibmdbpy import IdaDataBase, IdaDataFrame

# @hidden_cell
# This connection object is used to access your data and contains your credentials.
# You might want to remove those credentials before you share your notebook.
idadb_b17dd4d9dff84b519a9e627f3bec50c4 = IdaDataBase(dsn='DASHDB;Database=BLUDB;Hostname=dashdb-txn-sbox-yp-dal09-08.services.dal.bluemix.net;Port=50000;PROTOCOL=TCPIP;UID=sdc18363;PWD=g6tf3xjbqm+p2hxd')

sql_data = IdaDataFrame(idadb_b17dd4d9dff84b519a9e627f3bec50c4, 'SDC18363.PAXFEEDBACK').as_dataframe()
sql_data.head()

# You can close the database connection with the following code. Please keep the comment line with the @hidden_cell tag,
# because the close function displays parts of the credentials.
# @hidden_cell
# idadb_b17dd4d9dff84b519a9e627f3bec50c4.close()
# To learn more about the ibmdby package, please read the documentation: http://pythonhosted.org/ibmdbpy/


,Customer Type,Comments,Response,Categorisation,Sentiment,Sector,Flight No,Flight Date,Cabin Class,Seat No,Aircraft Version,Tail No,FFP Member,Gender,Age,Country
0,Passive,Amenity Kit: Comments,Très bien.,Compliment,Positive,MRU-CDG,NK0034,26/11/2018,ECONOMY,039E,359C,3BNBQ,NO,Male,26-34,France
1,Detractor,Amenity Kit: Comments,Not taking well kids,No adequate products,Negative,MRU-MAA,NK0746,26/11/2018,ECONOMY,None,343E,3BNBJ,NO,Male,35-44,Mauritius
2,Detractor,Amenity Kit: Comments,jusque dans le détail nous voyons la régressio...,Poor quality/ Very dissatisfied,Negative,BOM-MRU,NK0749,26/11/2018,ECONOMY,011C,343E,3BNBI,NO,Female,45-54,Mauritius
3,Detractor,Amenity Kit: Comments,I appreciate this kit in economy. It is a welc...,Compliment,Positive,SIN-MRU,NK0647,27/11/2018,ECONOMY,016B,332,3BNBM,NO,Female,55-64,Mauritius
4,Detractor,Amenity Kit: Comments,Nothing,None,Neutral,MRU-BOM,NK0748,27/11/2018,ECONOMY,026H,343C,3BNAU,YES,Male,45-54,India


In [2]:
# Data Inforamtion 
sql_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5347 entries, 0 to 5346
Data columns (total 16 columns):
Customer Type       5322 non-null object
Comments            5347 non-null object
Response            5334 non-null object
Categorisation      5300 non-null object
Sentiment           5179 non-null object
Sector              5347 non-null object
Flight No           5347 non-null object
Flight Date         5347 non-null object
Cabin Class         5347 non-null object
Seat No             4914 non-null object
Aircraft Version    5347 non-null object
Tail No             5317 non-null object
FFP Member          5317 non-null object
Gender              5224 non-null object
Age                 5185 non-null object
Country             5185 non-null object
dtypes: object(16)
memory usage: 668.5+ KB


In [3]:
# Drop null Records
sql_data.dropna(how='all',axis=0,inplace=True)
sql_data.shape

(5347, 16)

In [4]:
#upgrade Watson Packages
! pip install --upgrade "ibm-watson>=4.3.0"

     |████████████████████████████████| 368kB 10.9MB/s eta 0:00:01
     |████████████████████████████████| 204kB 28.3MB/s eta 0:00:01
  Stored in directory: /home/dsxuser/.cache/pip/wheels/73/21/91/61a59270c329be2edc947c69871c25c50bf7dd87ebbfe07cdc
  Stored in directory: /home/dsxuser/.cache/pip/wheels/6a/42/50/f96888116b329578304f9dda4693cef6f3e76e18272d22cb6c
Successfully built ibm-watson ibm-cloud-sdk-core


In [5]:
sql_data['Response'] = sql_data['Response'].astype(str)
sql_data.dtypes

Customer Type       object
Comments            object
Response            object
Categorisation      object
Sentiment           object
Sector              object
Flight No           object
Flight Date         object
Cabin Class         object
Seat No             object
Aircraft Version    object
Tail No             object
FFP Member          object
Gender              object
Age                 object
Country             object
dtype: object

In [6]:
# Add Watson Language Translators Service credentials for convert multiple language responses into English
from ibm_watson import LanguageTranslatorV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

authenticator = IAMAuthenticator('Bl81vpxYkB5y6DlhpISunsyjS0zOYME2JWRD0xWnCbgt')
service = LanguageTranslatorV3(version='2018-05-01',
   authenticator=authenticator
)
service.set_service_url('https://api.us-south.language-translator.watson.cloud.ibm.com/instances/78b55083-ee3f-4517-a209-1f07d880000d')

In [7]:
# Saving Model ID's in list ex:(Hindi-English) 
models_list=[]
size= len((service.list_models().get_result()).get('models'))
for i in range(0,size):
    models_list.append((service.list_models().get_result()).get('models')[i].get('model_id'))

In [11]:
from tqdm import tqdm

In [13]:
# Converting Multilanguage customer Responses to English
# saving outputs in List
identified_lang=[]
translated_lang=[]
identify_lang=list(sql_data['Response'])
for i in tqdm(identify_lang):
    detected_lang=(service.identify(i).get_result()).get('languages')[0].get('language')
    identified_lang.append(detected_lang)
    if (detected_lang != 'en' and '{}-en'.format(detected_lang) in models_list):
        tran=(service.translate(text=i,model_id='{}-en'.format(detected_lang)).get_result()).get('translations')[0].get('translation')
        translated_lang.append(tran)
    else:
        translated_lang.append(i)




  0%|          | 0/5347 [00:00<?, ?it/s]


  0%|          | 1/5347 [00:00<23:44,  3.75it/s]


  0%|          | 3/5347 [00:00<20:50,  4.27it/s]


  0%|          | 4/5347 [00:00<17:16,  5.15it/s]


  0%|          | 6/5347 [00:00<16:15,  5.48it/s]


  0%|          | 8/5347 [00:01<15:25,  5.77it/s]


  0%|          | 9/5347 [00:01<19:52,  4.48it/s]


  0%|          | 10/5347 [00:01<16:51,  5.28it/s]


  0%|          | 11/5347 [00:01<16:36,  5.35it/s]


  0%|          | 12/5347 [00:02<17:31,  5.07it/s]


  0%|          | 13/5347 [00:02<14:56,  5.95it/s]


  0%|          | 14/5347 [00:02<15:31,  5.73it/s]


  0%|          | 15/5347 [00:02<16:26,  5.40it/s]


  0%|          | 16/5347 [00:02<17:25,  5.10it/s]


  0%|          | 17/5347 [00:03<18:41,  4.75it/s]


  0%|          | 19/5347 [00:03<15:34,  5.70it/s]


  0%|          | 20/5347 [00:03<14:07,  6.29it/s]


  0%|          | 21/5347 [00:03<13:39,  6.50it/s]


  0%|          | 22/5347 [00:03<14:58,  5.93it/s]


  0%|          | 24/5347

  3%|▎         | 165/5347 [00:27<16:17,  5.30it/s]


  3%|▎         | 166/5347 [00:27<16:42,  5.17it/s]


  3%|▎         | 167/5347 [00:28<16:42,  5.17it/s]


  3%|▎         | 168/5347 [00:28<17:04,  5.06it/s]


  3%|▎         | 169/5347 [00:28<18:18,  4.71it/s]


  3%|▎         | 171/5347 [00:28<15:51,  5.44it/s]


  3%|▎         | 172/5347 [00:28<13:50,  6.23it/s]


  3%|▎         | 173/5347 [00:29<15:26,  5.59it/s]


  3%|▎         | 174/5347 [00:29<13:23,  6.43it/s]


  3%|▎         | 176/5347 [00:29<14:35,  5.91it/s]


  3%|▎         | 177/5347 [00:29<12:51,  6.70it/s]


  3%|▎         | 179/5347 [00:29<12:26,  6.92it/s]


  3%|▎         | 180/5347 [00:30<17:09,  5.02it/s]


  3%|▎         | 181/5347 [00:30<18:14,  4.72it/s]


  3%|▎         | 182/5347 [00:30<15:48,  5.44it/s]


  3%|▎         | 183/5347 [00:30<16:30,  5.21it/s]


  3%|▎         | 185/5347 [00:31<13:59,  6.15it/s]


  3%|▎         | 186/5347 [00:31<12:33,  6.85it/s]


  3%|▎         | 187/5347 [00:31<13:42,  6.27i

  6%|▌         | 332/5347 [00:53<14:41,  5.69it/s]


  6%|▌         | 333/5347 [00:53<17:44,  4.71it/s]


  6%|▌         | 334/5347 [00:54<15:13,  5.49it/s]


  6%|▋         | 335/5347 [00:54<15:39,  5.33it/s]


  6%|▋         | 336/5347 [00:54<13:31,  6.18it/s]


  6%|▋         | 337/5347 [00:54<12:20,  6.76it/s]


  6%|▋         | 338/5347 [00:54<15:15,  5.47it/s]


  6%|▋         | 339/5347 [00:54<17:29,  4.77it/s]


  6%|▋         | 341/5347 [00:55<14:50,  5.62it/s]


  6%|▋         | 343/5347 [00:55<12:49,  6.50it/s]


  6%|▋         | 345/5347 [00:55<11:18,  7.37it/s]


  6%|▋         | 346/5347 [00:55<14:22,  5.80it/s]


  6%|▋         | 347/5347 [00:55<12:58,  6.42it/s]


  7%|▋         | 348/5347 [00:56<14:21,  5.80it/s]


  7%|▋         | 350/5347 [00:56<12:25,  6.70it/s]


  7%|▋         | 351/5347 [00:56<11:51,  7.03it/s]


  7%|▋         | 352/5347 [00:56<11:27,  7.27it/s]


  7%|▋         | 353/5347 [00:56<11:15,  7.39it/s]


  7%|▋         | 355/5347 [00:56<10:53,  7.63i

  9%|▉         | 491/5347 [01:19<14:06,  5.73it/s]


  9%|▉         | 492/5347 [01:19<14:32,  5.56it/s]


  9%|▉         | 493/5347 [01:19<13:51,  5.84it/s]


  9%|▉         | 495/5347 [01:20<12:54,  6.27it/s]


  9%|▉         | 496/5347 [01:20<14:06,  5.73it/s]


  9%|▉         | 497/5347 [01:20<16:16,  4.97it/s]


  9%|▉         | 498/5347 [01:20<17:25,  4.64it/s]


  9%|▉         | 499/5347 [01:20<17:00,  4.75it/s]


  9%|▉         | 500/5347 [01:21<15:14,  5.30it/s]


  9%|▉         | 501/5347 [01:21<13:43,  5.88it/s]


  9%|▉         | 502/5347 [01:21<15:05,  5.35it/s]


  9%|▉         | 503/5347 [01:21<13:21,  6.05it/s]


  9%|▉         | 504/5347 [01:21<15:44,  5.13it/s]


  9%|▉         | 505/5347 [01:22<16:27,  4.90it/s]


  9%|▉         | 507/5347 [01:22<13:55,  5.79it/s]


 10%|▉         | 508/5347 [01:22<15:35,  5.17it/s]


 10%|▉         | 510/5347 [01:22<15:23,  5.24it/s]


 10%|▉         | 511/5347 [01:23<14:27,  5.57it/s]


 10%|▉         | 512/5347 [01:23<16:14,  4.96i

 12%|█▏        | 639/5347 [01:44<14:43,  5.33it/s]


 12%|█▏        | 640/5347 [01:44<15:30,  5.06it/s]


 12%|█▏        | 642/5347 [01:44<13:14,  5.93it/s]


 12%|█▏        | 643/5347 [01:44<14:08,  5.54it/s]


 12%|█▏        | 644/5347 [01:45<15:10,  5.17it/s]


 12%|█▏        | 645/5347 [01:45<15:50,  4.95it/s]


 12%|█▏        | 646/5347 [01:45<13:58,  5.61it/s]


 12%|█▏        | 648/5347 [01:45<12:19,  6.35it/s]


 12%|█▏        | 649/5347 [01:45<13:27,  5.82it/s]


 12%|█▏        | 650/5347 [01:45<14:01,  5.58it/s]


 12%|█▏        | 651/5347 [01:46<12:23,  6.31it/s]


 12%|█▏        | 652/5347 [01:46<11:41,  6.70it/s]


 12%|█▏        | 653/5347 [01:46<10:31,  7.43it/s]


 12%|█▏        | 654/5347 [01:46<09:56,  7.86it/s]


 12%|█▏        | 655/5347 [01:46<09:57,  7.86it/s]


 12%|█▏        | 656/5347 [01:46<11:58,  6.53it/s]


 12%|█▏        | 658/5347 [01:47<12:01,  6.50it/s]


 12%|█▏        | 659/5347 [01:47<14:02,  5.57it/s]


 12%|█▏        | 660/5347 [01:47<12:14,  6.38i

...

 88%|████████▊ | 4728/5347 [12:10<01:16,  8.05it/s]


 88%|████████▊ | 4729/5347 [12:10<01:29,  6.88it/s]


 88%|████████▊ | 4730/5347 [12:10<01:21,  7.54it/s]


 88%|████████▊ | 4731/5347 [12:10<01:15,  8.14it/s]


 88%|████████▊ | 4732/5347 [12:10<01:13,  8.37it/s]


 89%|████████▊ | 4734/5347 [12:10<01:09,  8.84it/s]


 89%|████████▊ | 4735/5347 [12:10<01:07,  9.12it/s]


 89%|████████▊ | 4736/5347 [12:11<01:09,  8.84it/s]


 89%|████████▊ | 4737/5347 [12:11<01:22,  7.39it/s]


 89%|████████▊ | 4738/5347 [12:11<01:34,  6.46it/s]


 89%|████████▊ | 4739/5347 [12:11<02:03,  4.94it/s]


 89%|████████▊ | 4740/5347 [12:11<02:03,  4.93it/s]


 89%|████████▊ | 4741/5347 [12:12<01:45,  5.73it/s]


 89%|████████▊ | 4742/5347 [12:12<01:52,  5.36it/s]


 89%|████████▊ | 4743/5347 [12:12<01:39,  6.06it/s]


 89%|████████▊ | 4745/5347 [12:12<01:35,  6.28it/s]


 89%|████████▉ | 4746/5347 [12:12<01:26,  6.93it/s]


 89%|████████▉ | 4747/5347 [12:13<01:39,  6.05it/s]


 89%|████████▉ | 4748/5347 [

 91%|█████████ | 4877/5347 [12:34<01:16,  6.14it/s]


 91%|█████████ | 4879/5347 [12:34<01:14,  6.24it/s]


 91%|█████████▏| 4880/5347 [12:34<01:13,  6.34it/s]


 91%|█████████▏| 4881/5347 [12:34<01:07,  6.88it/s]


 91%|█████████▏| 4882/5347 [12:35<01:02,  7.47it/s]


 91%|█████████▏| 4884/5347 [12:35<00:56,  8.14it/s]


 91%|█████████▏| 4886/5347 [12:35<01:01,  7.46it/s]


 91%|█████████▏| 4887/5347 [12:35<01:13,  6.23it/s]


 91%|█████████▏| 4889/5347 [12:35<01:04,  7.10it/s]


 91%|█████████▏| 4891/5347 [12:36<00:58,  7.81it/s]


 92%|█████████▏| 4893/5347 [12:36<00:54,  8.32it/s]


 92%|█████████▏| 4895/5347 [12:36<00:59,  7.59it/s]


 92%|█████████▏| 4896/5347 [12:36<00:56,  8.00it/s]


 92%|█████████▏| 4898/5347 [12:37<01:00,  7.48it/s]


 92%|█████████▏| 4900/5347 [12:37<01:02,  7.20it/s]


 92%|█████████▏| 4901/5347 [12:37<01:08,  6.49it/s]


 92%|█████████▏| 4903/5347 [12:37<01:02,  7.09it/s]


 92%|█████████▏| 4904/5347 [12:38<01:13,  6.05it/s]


 92%|█████████▏| 4905/5347 [

 94%|█████████▍| 5034/5347 [12:57<00:53,  5.87it/s]


 94%|█████████▍| 5035/5347 [12:57<00:47,  6.61it/s]


 94%|█████████▍| 5037/5347 [12:57<00:42,  7.34it/s]


 94%|█████████▍| 5038/5347 [12:58<00:40,  7.70it/s]


 94%|█████████▍| 5039/5347 [12:58<00:51,  5.96it/s]


 94%|█████████▍| 5040/5347 [12:58<00:55,  5.51it/s]


 94%|█████████▍| 5041/5347 [12:58<00:48,  6.33it/s]


 94%|█████████▍| 5042/5347 [12:58<00:50,  6.03it/s]


 94%|█████████▍| 5043/5347 [12:59<00:54,  5.55it/s]


 94%|█████████▍| 5044/5347 [12:59<00:50,  6.01it/s]


 94%|█████████▍| 5045/5347 [12:59<00:46,  6.44it/s]


 94%|█████████▍| 5046/5347 [12:59<00:44,  6.84it/s]


 94%|█████████▍| 5047/5347 [12:59<00:52,  5.73it/s]


 94%|█████████▍| 5048/5347 [12:59<00:56,  5.31it/s]


 94%|█████████▍| 5049/5347 [13:00<00:53,  5.57it/s]


 94%|█████████▍| 5050/5347 [13:00<00:46,  6.38it/s]


 94%|█████████▍| 5051/5347 [13:00<00:45,  6.55it/s]


 95%|█████████▍| 5053/5347 [13:00<00:40,  7.28it/s]


 95%|█████████▍| 5054/5347 [

 97%|█████████▋| 5178/5347 [13:20<00:53,  3.17it/s]


 97%|█████████▋| 5179/5347 [13:20<00:42,  3.98it/s]


 97%|█████████▋| 5180/5347 [13:20<00:34,  4.82it/s]


 97%|█████████▋| 5181/5347 [13:20<00:29,  5.56it/s]


 97%|█████████▋| 5182/5347 [13:21<00:32,  5.12it/s]


 97%|█████████▋| 5183/5347 [13:21<00:33,  4.92it/s]


 97%|█████████▋| 5184/5347 [13:21<00:28,  5.74it/s]


 97%|█████████▋| 5185/5347 [13:21<00:36,  4.47it/s]


 97%|█████████▋| 5187/5347 [13:22<00:32,  4.96it/s]


 97%|█████████▋| 5188/5347 [13:22<00:27,  5.77it/s]


 97%|█████████▋| 5189/5347 [13:22<00:24,  6.47it/s]


 97%|█████████▋| 5190/5347 [13:22<00:22,  6.85it/s]


 97%|█████████▋| 5191/5347 [13:22<00:20,  7.45it/s]


 97%|█████████▋| 5192/5347 [13:22<00:19,  7.90it/s]


 97%|█████████▋| 5193/5347 [13:22<00:24,  6.34it/s]


 97%|█████████▋| 5194/5347 [13:22<00:21,  7.03it/s]


 97%|█████████▋| 5195/5347 [13:23<00:20,  7.57it/s]


 97%|█████████▋| 5196/5347 [13:23<00:19,  7.82it/s]


 97%|█████████▋| 5197/5347 [

100%|█████████▉| 5329/5347 [13:43<00:02,  8.03it/s]


100%|█████████▉| 5330/5347 [13:43<00:02,  8.16it/s]


100%|█████████▉| 5331/5347 [13:43<00:01,  8.42it/s]


100%|█████████▉| 5332/5347 [13:43<00:01,  8.80it/s]


100%|█████████▉| 5333/5347 [13:43<00:01,  8.91it/s]


100%|█████████▉| 5334/5347 [13:43<00:01,  7.01it/s]


100%|█████████▉| 5335/5347 [13:43<00:02,  5.91it/s]


100%|█████████▉| 5336/5347 [13:44<00:01,  6.68it/s]


100%|█████████▉| 5337/5347 [13:44<00:01,  7.18it/s]


100%|█████████▉| 5338/5347 [13:44<00:01,  7.06it/s]


100%|█████████▉| 5339/5347 [13:44<00:01,  7.03it/s]


100%|█████████▉| 5340/5347 [13:44<00:01,  6.18it/s]


100%|█████████▉| 5342/5347 [13:44<00:00,  6.23it/s]


100%|█████████▉| 5343/5347 [13:45<00:00,  5.67it/s]


100%|█████████▉| 5345/5347 [13:45<00:00,  6.56it/s]


100%|█████████▉| 5346/5347 [13:45<00:00,  7.21it/s]


100%|██████████| 5347/5347 [13:45<00:00,  7.69it/s]

In [14]:
# inserting Identified Language ID & Translated_Language in to Dataframe
sql_data.insert(2, "Identified_Language", identified_lang, True)
sql_data.insert(3, "Translated_Language", translated_lang, True)

In [15]:
# Checking Shape of dataframe(Number of records & Coulmns Available in Dataframe)
sql_data.shape

(5347, 18)

In [16]:
# Sample View of Dataframe
sql_data.head()

,Customer Type,Comments,Identified_Language,Translated_Language,Response,Categorisation,Sentiment,Sector,Flight No,Flight Date,Cabin Class,Seat No,Aircraft Version,Tail No,FFP Member,Gender,Age,Country
0,Passive,Amenity Kit: Comments,fr,Very well.,Très bien.,Compliment,Positive,MRU-CDG,NK0034,26/11/2018,ECONOMY,039E,359C,3BNBQ,NO,Male,26-34,France
1,Detractor,Amenity Kit: Comments,en,Not taking well kids,Not taking well kids,No adequate products,Negative,MRU-MAA,NK0746,26/11/2018,ECONOMY,None,343E,3BNBJ,NO,Male,35-44,Mauritius
2,Detractor,Amenity Kit: Comments,fr,Up to the detail we see the regression of Air ...,jusque dans le détail nous voyons la régressio...,Poor quality/ Very dissatisfied,Negative,BOM-MRU,NK0749,26/11/2018,ECONOMY,011C,343E,3BNBI,NO,Female,45-54,Mauritius
3,Detractor,Amenity Kit: Comments,en,I appreciate this kit in economy. It is a welc...,I appreciate this kit in economy. It is a welc...,Compliment,Positive,SIN-MRU,NK0647,27/11/2018,ECONOMY,016B,332,3BNBM,NO,Female,55-64,Mauritius
4,Detractor,Amenity Kit: Comments,en,Nothing,Nothing,None,Neutral,MRU-BOM,NK0748,27/11/2018,ECONOMY,026H,343C,3BNAU,YES,Male,45-54,India


In [17]:
!pip install ibm_db_sa

In [20]:
import ibm_db
import pandas as pd
from sqlalchemy import create_engine
import sqlalchemy

In [18]:

dsn_driver = "IBM DB2 ODBC DRIVER"
dsn_database = "BLUDB"
dsn_hostname= "dashdb-txn-sbox-yp-dal09-08.services.dal.bluemix.net"
dsn_port="50000"
dsn_protocol="TCPIP"
dsn_uid="sdc18363"
dsn_pwd="g6tf3xjbqm+p2hxd"

In [21]:
engine = create_engine('ibm_db_sa://'+ dsn_uid + ':' + dsn_pwd + '@'+dsn_hostname+':'+dsn_port+'/' + dsn_database )

In [24]:
sql_data.to_sql('translations', engine,index=False)